In [1]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os
import platform
import shutil
import networkx as nx



/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# read the network topology that is HDMA hydrolakes global created by the repository
ds = xr.open_dataset('/Users/shg096/Desktop/mizuRoute_HDMA_Global/data/Network_topology_HDMA_HydroLake_v1.nc')
ds

/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'rasterio' loading failed:
[Errno 2] No such file or directory: '/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/numpy-1.21.1.dist-info/METADATA'
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:    (n: 298277)
Coordinates:
    ID         (n) float64 ...
Dimensions without coordinates: n
Data variables:
    ID_next    (n) float64 ...
    length     (n) float64 ...
    area       (n) float64 ...
    lake       (n) float64 ...
    endorheic  (n) float64 ...
    slope      (n) float64 ...
    S_0        (n) float64 ...
    S_max      (n) float64 ...
    Coeff      (n) float64 ...
    power      (n) float64 ...
    lake_type  (n) float64 ...
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Fri Jul 14 14:50:18 2023
    source:       Written by test script of utilities (https://github.com/She...

In [4]:
target_ID = 3044094 # segment ID downstream of lake victoria

# convert ds back to df
df = ds.to_dataframe()

# drop columns from df is available
df = df.drop(['target_vol','lake_type_for_target_vol'], axis=1, errors='ignore')

# make the string out of ID and then slice the dataframe based on 2 (africa) and 7 lakes
df ['ID_str'] = df['ID'].astype(str)
df = df[df['ID_str'].astype(str).str.startswith(('3', '7'))]

# create the riv graph
riv_graph = nx.from_pandas_edgelist(df,\
                                    source='ID',\
                                    target='ID_next',\
                                    create_using=nx.DiGraph)
nodes = nx.ancestors(riv_graph, target_ID)
nodes = np.array(list(nodes))
nodes = np.append(nodes, target_ID)

# now slice the ds based on existing nodes
idx = np.where(np.isin(ds['ID'].values, nodes))
idx = np.array(idx).flatten()
ds_slice = ds.isel(n=idx)

# find the location of target segment and add -9999 as downstream
idx = np.where(np.isin(ds_slice['ID'].values, target_ID))
idx = np.array(idx).flatten()
ds_slice ['ID_next'][idx] = -9999

ds_slice

<xarray.Dataset>
Dimensions:    (n: 290)
Coordinates:
    ID         (n) float64 3e+06 3e+06 3e+06 ... 7.008e+06 7.008e+06 7.1e+06
Dimensions without coordinates: n
Data variables:
    ID_next    (n) float64 3.022e+06 7.001e+06 3.033e+06 ... 3.035e+06 3.026e+06
    length     (n) float64 347.9 1.0 2.229e+03 4.591e+03 ... 1.0 1.0 1.0 1.0
    area       (n) float64 2.486e+08 2.444e+08 2.512e+08 ... 2.163e+07 4.41e+06
    lake       (n) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
    endorheic  (n) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    slope      (n) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    S_0        (n) float64 1e+06 1e+06 1e+06 ... 3.261e+08 4.9e+08 1.829e+07
    S_max      (n) float64 1.244e+09 1.223e+09 1.257e+09 ... 5.982e+08 4.034e+07
    Coeff      (n) float64 0.01 0.01 0.01 0.01 0.01 ... 0.01 0.01 0.01 0.01 0.01
    power      (n) float64 1.5 1.5 1.5 1.5 1.5 1.5 ... 1.5 1.5 1.5 1.5 1.5 1.5
    lake_type  (n) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Fri Jul 14 14:50:18 2023
    source:       Written by test script of utilities (https://github.com/She...

In [ ]:
# save the network topology
df = pd.read_csv('../data/network_topology/Saskatoon_NTOPO_no_lake.csv')

df = df.set_index('ID')

ds = df.to_xarray()

ds.attrs['Conventions'] = 'CF-1.6'
ds.attrs['License']     = 'The data were written by Shervan Gharari. They are under GPL.'
ds.attrs['history']     = 'Created ' + time.ctime(time.time())
ds.attrs['source']      = 'Written by test script of utilities (https://github.com/ShervanGharari/utility-codes'

var_info = {'ID':        {'long_name': 'ID',        'unit': '-'},
            'ID_next':   {'long_name': 'ID next',   'unit': '-'},
            'length':    {'long_name': 'length',    'unit': 'm'},
            'area':      {'long_name': 'area',      'unit': 'm**2'},
            'slope':     {'long_name': 'slope',     'unit': 'm m**-1'}}

var_encoding = {'ID':        {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                'ID_next':   {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                'length':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                'area':      {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                'slope':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9}}

for key in var_info.keys():
    for key1 in var_info[key].keys():
        print(var_info[key][key1])
        ds[key].attrs[key1] = var_info[key][key1]
        print(ds[key].attrs[key1])


if os.path.isfile('../data/network_topology/Network_topology_merit_no_lake.nc'):
    os.remove('../data/network_topology/Network_topology_merit_no_lake.nc')

ds = ds.rename_dims({'ID': 'n'})
ds.to_netcdf('../data/network_topology/Network_topology_merit_no_lake.nc',\
             encoding = var_encoding)

#
ds = xr.open_dataset('../data/network_topology/Network_topology_merit_no_lake.nc')

ds